## Production Planning LP Problem

In [3]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import pandas as pd
import numpy as np

PMG Corp is one of the leading industrial stampers in the city in which you operate a consulting firm for sales and operation planning optimization.  PMG has contracted with you for assistance in planning their production schedule for the upcoming Quarter 1 (13 weeks).  The company has orders for 15 different parts that can be both be produced on one of two production lines: line A or line B.  Table 1 summarizes the orders for each part number that must be produced in the coming quarter along with their production rates and costs on each line, and the cost of subcontracting each order.  Note that the first 4 orders can only be produced on line A.  Assume that any portion of an order can be subcontracted.  

PMG currently has 15 Line A machines and 80 Line B machines.  Each machine operates 24 hours a day, 7 days a week, except for a scheduled downtime of 2 hours per week for maintenance.  

**Table 1**  

|Part Number|Q1 Sales Demand (parts) |Line A Parts/Hr|Line A Cost/Part|Line B Parts/Hr|Line B Cost/Part|Subcontract Cost/Part|
|:---------:|:----------------------:|:-------------:|:--------------:|:-------------:|:--------------:|:-------------------:|
|1          |72,190                  |4.510          |\\$2.66           |na             |na            |\\$2.77              |
|2          |235,765                 |4.796          |\\$2.55           |na             |na            |\\$2.73              | 
|3          |245,125                 |4.629          |\\$2.64           |na             |na            |\\$2.85              | 
|4          |242,565                 |4.256          |\\$2.56           |na             |na            |\\$2.73              | 
|5          |93,252                  |5.145          |\\$1.61           |5.426          |\\$1.60       |\\$1.76              | 
|6          |57,996                  |3.806          |\\$1.62           |3.935          |\\$1.61       |\\$1.76              | 
|7          |190,128                 |4.168          |\\$1.64           |4.316          |\\$1.61       |\\$1.76              | 
|8          |167,670                 |5.251          |\\$1.48           |5.356          |\\$1.47       |\\$1.59              | 
|9          |294,790                 |5.223          |\\$1.50           |5.277          |\\$1.50       |\\$1.71              | 
|10         |96,355                  |5.216          |\\$1.44           |5.419          |\\$1.42       |\\$1.63              | 
|11         |249,448                 |3.744          |\\$1.64           |3.835          |\\$1.64       |\\$1.80              | 
|12         |100,354                 |4.157          |\\$1.57           |4.291          |\\$1.56       |\\$1.78              | 
|13         |263,163                 |4.422          |\\$1.49           |4.558          |\\$1.48       |\\$1.63              | 
|14         |245,189                 |5.281          |\\$1.31           |5.353          |\\$1.30       |\\$1.44              | 
|15         |89,239                  |4.222          |\\$1.51           |4.288          |\\$1.50       |\\$1.69              | 


Create a linear optimization model to meet sales demand at the lowest possible cost and find the optimal production plan and associated cost.

How would the total cost be impacted if one of the Line A machines broke and could not be used for the entire quarter?

If all Part Numbers 5 - 15 all sell for the same amount, which part number should the salesforce of PMG try to sell more of?  Why?

In [32]:
model = LpProblem("Producion_Planning", LpMinimize)

In [43]:
Prod = pd.read_csv('ProductionData.csv')
Demand = pd.read_csv('DemandData.csv')
Demand = Demand.set_index('Part_Number')

Prod

,Part_Number,Source,Parts_Per_Hour,Cost_Per_Part
0,1,LineA,4.51,2.66
1,2,LineA,4.796,2.55
2,3,LineA,4.629,2.64
3,4,LineA,4.256,2.56
4,5,LineA,5.145,1.61
5,6,LineA,3.806,1.62
6,7,LineA,4.168,1.64
7,8,LineA,5.251,1.48
8,9,LineA,5.223,1.5
9,10,LineA,5.216,1.44


In [44]:
Prod = Prod.set_index(['Part_Number', 'Source'])
Prod.head(10)

,,Parts_Per_Hour,Cost_Per_Part
Part_Number,Source,,
1,LineA,4.51,2.66
2,LineA,4.796,2.55
3,LineA,4.629,2.64
4,LineA,4.256,2.56
5,LineA,5.145,1.61
6,LineA,3.806,1.62
7,LineA,4.168,1.64
8,LineA,5.251,1.48
9,LineA,5.223,1.5


In [45]:
Prod.index.levels

FrozenList([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ['LineA', 'LineB', 'Subcontract']])

In [46]:
## Add column for Hours/Part

Prod['Hours_Per_Part'] = [0 if x == 'na' else pd.to_numeric(x) ** -1 for x in Prod['Parts_Per_Hour']]
Prod = Prod.replace('na',0)    
Prod['Cost_Per_Part'] = Prod['Cost_Per_Part'].apply(pd.to_numeric)
    
Prod.head(10)

,,Parts_Per_Hour,Cost_Per_Part,Hours_Per_Part
Part_Number,Source,,,
1,LineA,4.51,2.66,0.221729
2,LineA,4.796,2.55,0.208507
3,LineA,4.629,2.64,0.216029
4,LineA,4.256,2.56,0.234962
5,LineA,5.145,1.61,0.194363
6,LineA,3.806,1.62,0.262743
7,LineA,4.168,1.64,0.239923
8,LineA,5.251,1.48,0.190440
9,LineA,5.223,1.50,0.191461


In [47]:
##Calculate hours available for each source. 


Line_Hours = {'LineA': 15, 'LineB': 80, 'Subcontract': 0}

Prod_Hours_Day = 24
Prod_Days_Week = 7
Prod_Maint_Hours_Week = 2
Prod_Weeks_Qtr = 13

Hours_Qtr = Prod_Weeks_Qtr * (Prod_Hours_Day * Prod_Days_Week - Prod_Maint_Hours_Week)

Line_Hours.update((line, hours * Hours_Qtr) for line, hours in Line_Hours.items())
Line_Hours

{'LineA': 32370, 'LineB': 172640, 'Subcontract': 0}

In [48]:
## Note about multiindex
Prod.index.levels[1]

Index(['LineA', 'LineB', 'Subcontract'], dtype='object', name='Source')

In [49]:
# Construct Decision Variables

Sources = np.unique(Prod.index.levels[1])
Parts = np.unique(Prod.index.levels[0])

ProdPlan = LpVariable.dicts("qty", ((p,s) for p in Parts for s in Sources))
ProdPlan

{(1, 'LineA'): qty_(1,_'LineA'),
 (1, 'LineB'): qty_(1,_'LineB'),
 (1, 'Subcontract'): qty_(1,_'Subcontract'),
 (2, 'LineA'): qty_(2,_'LineA'),
 (2, 'LineB'): qty_(2,_'LineB'),
 (2, 'Subcontract'): qty_(2,_'Subcontract'),
 (3, 'LineA'): qty_(3,_'LineA'),
 (3, 'LineB'): qty_(3,_'LineB'),
 (3, 'Subcontract'): qty_(3,_'Subcontract'),
 (4, 'LineA'): qty_(4,_'LineA'),
 (4, 'LineB'): qty_(4,_'LineB'),
 (4, 'Subcontract'): qty_(4,_'Subcontract'),
 (5, 'LineA'): qty_(5,_'LineA'),
 (5, 'LineB'): qty_(5,_'LineB'),
 (5, 'Subcontract'): qty_(5,_'Subcontract'),
 (6, 'LineA'): qty_(6,_'LineA'),
 (6, 'LineB'): qty_(6,_'LineB'),
 (6, 'Subcontract'): qty_(6,_'Subcontract'),
 (7, 'LineA'): qty_(7,_'LineA'),
 (7, 'LineB'): qty_(7,_'LineB'),
 (7, 'Subcontract'): qty_(7,_'Subcontract'),
 (8, 'LineA'): qty_(8,_'LineA'),
 (8, 'LineB'): qty_(8,_'LineB'),
 (8, 'Subcontract'): qty_(8,_'Subcontract'),
 (9, 'LineA'): qty_(9,_'LineA'),
 (9, 'LineB'): qty_(9,_'LineB'),
 (9, 'Subcontract'): qty_(9,_'Subcontract'),
 

In [50]:
Sources[0]

'LineA'

In [51]:
Parts[9]

10

In [52]:
##A note on indexing

p = Parts[9]
s = Sources[0]
Prod.loc[(p, s), 'Cost_Per_Part']

1.44

In [53]:
Prod.loc[(10, 'LineA'), 'Cost_Per_Part']

1.44

In [54]:
##This is the equivalent R syntax, for reference
#Prod[Prod$Part == 14 & Prod$Source == 'LineA', 'Cost_Per_Part']

In [55]:
# Add the objective function to the model
model += lpSum(Prod.loc[(p, s), 'Cost_Per_Part'] * ProdPlan[p, s] for p in Parts for s in Sources)


In [56]:
# Add the capacity constraints to the model

for s in Sources:
    model +=  (lpSum(ProdPlan[p, s] * Prod.loc[(p, s), 'Hours_Per_Part'] for p in Parts) <= Line_Hours[s],s)
    


In [57]:
model

Producion_Planning:
MINIMIZE
2.66*qty_(1,_'LineA') + 2.77*qty_(1,_'Subcontract') + 1.44*qty_(10,_'LineA') + 1.42*qty_(10,_'LineB') + 1.63*qty_(10,_'Subcontract') + 1.64*qty_(11,_'LineA') + 1.64*qty_(11,_'LineB') + 1.8*qty_(11,_'Subcontract') + 1.57*qty_(12,_'LineA') + 1.56*qty_(12,_'LineB') + 1.78*qty_(12,_'Subcontract') + 1.49*qty_(13,_'LineA') + 1.48*qty_(13,_'LineB') + 1.63*qty_(13,_'Subcontract') + 1.31*qty_(14,_'LineA') + 1.3*qty_(14,_'LineB') + 1.44*qty_(14,_'Subcontract') + 1.51*qty_(15,_'LineA') + 1.5*qty_(15,_'LineB') + 1.69*qty_(15,_'Subcontract') + 2.55*qty_(2,_'LineA') + 2.73*qty_(2,_'Subcontract') + 2.64*qty_(3,_'LineA') + 2.85*qty_(3,_'Subcontract') + 2.56*qty_(4,_'LineA') + 2.73*qty_(4,_'Subcontract') + 1.61*qty_(5,_'LineA') + 1.6*qty_(5,_'LineB') + 1.76*qty_(5,_'Subcontract') + 1.62*qty_(6,_'LineA') + 1.61*qty_(6,_'LineB') + 1.76*qty_(6,_'Subcontract') + 1.64*qty_(7,_'LineA') + 1.61*qty_(7,_'LineB') + 1.76*qty_(7,_'Subcontract') + 1.48*qty_(8,_'LineA') + 1.47*qty_(8,_'L

In [58]:
Demand

,Q1_Demand
Part_Number,
1,72190
2,235765
3,245125
4,242565
5,93252
6,57996
7,190128
8,167670
9,294790


In [59]:
## Add the demand constraints to the model

for p in Parts:
    model += (lpSum(ProdPlan[p, s] for s in Sources) == Demand.loc[p],p)



In [60]:
## Add infeasability constraints for Parts 1 - 4 on Line B

model += ProdPlan[(1, 'LineB')] == 0
model += ProdPlan[(2, 'LineB')] == 0
model += ProdPlan[(3, 'LineB')] == 0
model += ProdPlan[(4, 'LineB')] == 0




In [61]:
## Add lower bound constraints to the model

for p in Parts :
    for s in Sources :
        model += (ProdPlan[p, s] >=0, 'low_bound_' + str(p) + '_' + str(s))

In [62]:
model

Producion_Planning:
MINIMIZE
2.66*qty_(1,_'LineA') + 2.77*qty_(1,_'Subcontract') + 1.44*qty_(10,_'LineA') + 1.42*qty_(10,_'LineB') + 1.63*qty_(10,_'Subcontract') + 1.64*qty_(11,_'LineA') + 1.64*qty_(11,_'LineB') + 1.8*qty_(11,_'Subcontract') + 1.57*qty_(12,_'LineA') + 1.56*qty_(12,_'LineB') + 1.78*qty_(12,_'Subcontract') + 1.49*qty_(13,_'LineA') + 1.48*qty_(13,_'LineB') + 1.63*qty_(13,_'Subcontract') + 1.31*qty_(14,_'LineA') + 1.3*qty_(14,_'LineB') + 1.44*qty_(14,_'Subcontract') + 1.51*qty_(15,_'LineA') + 1.5*qty_(15,_'LineB') + 1.69*qty_(15,_'Subcontract') + 2.55*qty_(2,_'LineA') + 2.73*qty_(2,_'Subcontract') + 2.64*qty_(3,_'LineA') + 2.85*qty_(3,_'Subcontract') + 2.56*qty_(4,_'LineA') + 2.73*qty_(4,_'Subcontract') + 1.61*qty_(5,_'LineA') + 1.6*qty_(5,_'LineB') + 1.76*qty_(5,_'Subcontract') + 1.62*qty_(6,_'LineA') + 1.61*qty_(6,_'LineB') + 1.76*qty_(6,_'Subcontract') + 1.64*qty_(7,_'LineA') + 1.61*qty_(7,_'LineB') + 1.76*qty_(7,_'Subcontract') + 1.48*qty_(8,_'LineA') + 1.47*qty_(8,_'L

In [63]:
model.solve()

1

In [64]:
model.solve()

LpStatus[model.status]

'Optimal'

In [65]:
model.objective.value()

5097706.322980001

In [66]:
for v in model.variables(): print(f"{v.name}: {v.varValue}")

qty_(1,_'LineA'): 0.0
qty_(1,_'LineB'): 0.0
qty_(1,_'Subcontract'): 72190.0
qty_(10,_'LineA'): 0.0
qty_(10,_'LineB'): 96355.0
qty_(10,_'Subcontract'): 0.0
qty_(11,_'LineA'): 0.0
qty_(11,_'LineB'): 0.0
qty_(11,_'Subcontract'): 249448.0
qty_(12,_'LineA'): 0.0
qty_(12,_'LineB'): 100354.0
qty_(12,_'Subcontract'): 0.0
qty_(13,_'LineA'): 0.0
qty_(13,_'LineB'): 0.0
qty_(13,_'Subcontract'): 263163.0
qty_(14,_'LineA'): 0.0
qty_(14,_'LineB'): 201367.19
qty_(14,_'Subcontract'): 43821.812
qty_(15,_'LineA'): 0.0
qty_(15,_'LineB'): 89239.0
qty_(15,_'Subcontract'): 0.0
qty_(2,_'LineA'): 0.0
qty_(2,_'LineB'): 0.0
qty_(2,_'Subcontract'): 235765.0
qty_(3,_'LineA'): 149840.73
qty_(3,_'LineB'): 0.0
qty_(3,_'Subcontract'): 95284.27
qty_(4,_'LineA'): 0.0
qty_(4,_'LineB'): 0.0
qty_(4,_'Subcontract'): 242565.0
qty_(5,_'LineA'): 0.0
qty_(5,_'LineB'): 93252.0
qty_(5,_'Subcontract'): 0.0
qty_(6,_'LineA'): 0.0
qty_(6,_'LineB'): 0.0
qty_(6,_'Subcontract'): 57996.0
qty_(7,_'LineA'): 0.0
qty_(7,_'LineB'): 0.0
qty_(7

In [34]:
for c in model.constraints.values(): print(f"{c.name}: {c.pi}")

LineA: -0.97209
LineB: -0.74942
Subcontract: 0.0
1: 2.77
2: 2.73
3: 2.85
4: 2.73
5: 1.7380656
6: 1.76
7: 1.76
8: 1.59
9: 1.6420163
10: 1.5582949
11: 1.8
12: 1.7346493
13: 1.63
14: 1.44
15: 1.6747715
None: -2.77
None: -2.73
None: -2.85
None: -2.73
low_bound_1_LineA: 0.10554102
low_bound_1_LineB: 0.0
low_bound_1_Subcontract: 0.0
low_bound_2_LineA: 0.022687656
low_bound_2_LineB: 0.0
low_bound_2_Subcontract: 0.0
low_bound_3_LineA: 0.0
low_bound_3_LineB: 0.0
low_bound_3_Subcontract: 0.0
low_bound_4_LineA: 0.058404605
low_bound_4_LineB: 0.0
low_bound_4_Subcontract: 0.0
low_bound_5_LineA: 0.06087319
low_bound_5_LineB: 0.0
low_bound_5_Subcontract: 0.021934414
low_bound_6_LineA: 0.11540988
low_bound_6_LineB: 0.040449809
low_bound_6_Subcontract: 0.0
low_bound_7_LineA: 0.11322697
low_bound_7_LineB: 0.023637627
low_bound_7_Subcontract: 0.0
low_bound_8_LineA: 0.075124738
low_bound_8_LineB: 0.019921583
low_bound_8_Subcontract: 0.0
low_bound_9_LineA: 0.044100877
low_bound_9_LineB: 0.0
low_bound_9_Sub

In [67]:
model.constraints['LineA'].pi

-0.97209

How would the total cost be impacted if one of the Line A machines broke and could not be used for the entire quarter?

> Shadow price of Line A Capacity constraint = -.97209  
> If 2158 hours are lost on Line A, cost will increase by -1 * 2158 * -.97209 = \\$2,098.

If all Part Numbers 5 - 15 all sell for the same amount, which part number should the salesforce of PMG try to sell more of?  Why?  
> Minimum shadow prrice for all parts = \\$1.44 for Part# 14.  
> Relaxing the order constraint (i.e., taking more orders) for Part 14 will increase costs the least.